Movie Scripts

In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn
from pytube import YouTube

In [249]:
# Store data in lists
names = []
runtimes = []
profile_pages = []
# storylines = []
release_dates = []
budgets = []
plot_synopsis = []
production_companies = []
spec_eff_comps = []
gross = []
stars = []
directors = []
screen_writers = []
metascores = []
genres = []
mpaa_ratings = []
imdb_ratings = []


# pages = [str(i) for i in range(1,10000,250)]
pages = ['1']

#Prepare monitoring of loop
start_time = time()
requests = 0
#for every page
for page in pages:

    #make get request
    response = get("https://www.imdb.com/search/title?title_type=feature&release_date=2014-01-01,2018-12-31&languages=en&count=50&start=" + page)

    #pause the loop
    sleep(randint(8,15))

    #monitor requests
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    #throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    #break the loop if the number of requests is greater than expected
    if requests > 72:
        warn('Number of requests was greater than expected.')
        break

    # parse the content of request
    page_html = BeautifulSoup(response.text, 'html.parser')

    #select all 250 movie containers from a single page
    mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    del mv_containers[:25]

    # Extract data from indiv. movie containers
    for container in mv_containers:
        if container.find('div', class_ = 'ratings-metascore') is not None:
            #Name
            name = container.h3.a.text
            names.append(name)
            
            #Runtime
            runtime = container.find('span', class_ = 'runtime').text
            runtimes.append(runtime)
                        
            #Stars
            movie_stars = []
            stars_x = container.findAll('div')[3].findAll('p')[2].findAll('a')
            del stars_x[0]
            for star in stars_x:
                movie_stars.append(star.text)
            stars.append(movie_stars)

            #Profile
            sleep(randint(1,3))
            profile = container.h3.a['href']
            profile_pages.append(profile)

            details = get("https://www.imdb.com/" + profile)
            # parse the content of request
            details_html = BeautifulSoup(details.text, 'html.parser')

            #save storyline
#             storyline = details_html.find("div", {"id": "titleStoryLine"}).div.span.text
#             storylines.append(storyline)
            
            try:
                #Budget
                bo_budget = details_html.find(text='Budget:').parent.findNext('span').decompose()
                budget = details_html.find(text='Budget:').parent.parent.text.strip()
                budgets.append(budget[7:].replace('$', '').replace(',', ''))
            except:
                budgets.append("null")
                    
            #Release Dates
            html_text = "Release Date:"
            details_html.find(text='Release Date:').parent.findNext('span').decompose()
            date = details_html.find(text='Release Date:').parent.parent.text.strip()
            release_dates.append(re.sub(r'\([^)]*\)', '', date[14:])[:-1])
            
            
            #Production Companies
            production_comps = []
            sleep(randint(1,3))
            production_link = profile[:-15]
            production = get("https://www.imdb.com/" + production_link + "companycredits")
            production_html = BeautifulSoup(production.text, 'html.parser')
            comps_x = production_html.find("div", {"id": "company_credits_content"}).ul.findAll('a')
            for comp in comps_x:
                production_comps.append(comp.text)
            production_companies.append(production_comps)
            
            #Special Effects Companies
            effects_comps = []
            comps_x = effects_html.find("h4", {"id": "specialEffects"}).findNext('ul').findAll('li')
            for comp in comps_x:
                effects_comps.append(comp.a.text)
            spec_eff_comps.append(effects_comps)
            
            #Director
            director = container.findAll('div')[3].findAll('p')[2].findAll('a')[0].text
            directors.append(director)


            synopsis_link = profile[:-15]
            sleep(randint(1,3))
            synopsis = get("https://www.imdb.com/" + synopsis_link + "plotsummary")
            synopsis_html = BeautifulSoup(synopsis.text, 'html.parser')

            plot_synopsis_content = synopsis_html.find("ul", {"id": "plot-synopsis-content"}).li.text
            plot_synopsis_content = plot_synopsis_content.strip()
            plot_synopsis.append(plot_synopsis_content)
            
            writers = []
            credit_containers = full_credits_html.find_all('table', class_ = 'simpleTable simpleCreditsTable')
            writers_x = credit_containers[1].tbody.findAll('tr')
            for writer in writers_x:
                if writer.find('td', attrs = {'colspan':'3'}):
                    1==1
                else:
                    answer = writer.find('td', class_ = 'name').text
                    writers.append(answer.strip())
            screen_writers.append(writers)
            
            
            #Metascore
            metascore = container.find('span', class_ = 'metascore')
            metascore = int(metascore.text)
            metascores.append(metascore)

            
            #Genre
            genre = container.p.find('span', class_ = 'genre').text.strip()
            genres.append(genre)
            
            if container.p.find('span', class_ = 'certificate') is not None:
                #MPAA rating
                mpaa = container.p.find('span', class_ = 'certificate').text
                mpaa_ratings.append(mpaa)
            else:
                na = "N/A"
                mpaa_ratings.append(na)
                
            #IMDb rating
            imdb = float(container.strong.text)
            imdb_ratings.append(imdb)
            
            try:
                #Gross
                bo_gross = container.findAll('div')[3].findAll('p')[3].findAll('span')[4]
                bo_gross = bo_gross['data-value']
                bo_gross = re.sub("[^\d\.]", "", bo_gross)
                gross.append(int(bo_gross))
            except:
                gross.append("null")
                
                
#             #Trailer
            movie_name = name
            response = get("https://www.google.com/search?q={}+Trailer".format(movie_name.replace(" ", "+")))
            html_google = BeautifulSoup(response.text, 'html.parser')


            txt = html_google.find('div', class_='g').text
            x = re.findall("(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?",txt)
            link = "{}://{}{}".format(x[0][0], x[0][1], x[0][2])
            yt = pytube.YouTube(link)
            stream = yt.streams.first()
            stream.download("C:\\Users\Brian\Desktop\IMDb_Scrape\Movie_Trailers", "{}_trailer".format(movie_name.replace(" ", "_")))
            

Request:1; Frequency: 0.08802418680644135 requests/s


In [250]:
print("Names Length: {}".format(len(names)))
print("Length: {}".format(len(runtimes)))
print("Profile: {}".format(len(profile_pages)))
# print("Storyline: {}".format(len(storylines)))
print("Spe. Eff.: {}".format(len(spec_eff_comps)))
print("Date: {}".format(len(release_dates)))
print("Budget: {}".format(len(budgets)))
print("Synopsis: {}".format(len(plot_synopsis)))
print("Pro. Comp: {}".format(len(production_companies)))
print("Gross Length: {}".format(len(gross)))
print("Stars: {}".format(len(stars)))
print("Director Length: {}".format(len(directors)))
print("Screen Writers Length: {}".format(len(screen_writers)))
print("Metascore: {}".format(len(metascores)))
print("Genre: {}".format(len(genres)))
print("MPAA: {}".format(len(mpaa_ratings)))
print("IMDB: {}".format(len(imdb_ratings)))

Names Length: 23
Length: 23
Profile: 23
Spe. Eff.: 23
Date: 23
Budget: 23
Synopsis: 23
Pro. Comp: 23
Gross Length: 23
Stars: 23
Director Length: 23
Screen Writers Length: 23
Metascore: 23
Genre: 23
MPAA: 23
IMDB: 23


In [251]:
movie_ratings = pd.DataFrame({'movie': names,
                              'length': runtimes,
#                               'profile': profile_pages,
#                               'storyline': storylines,
                              'date': release_dates,
                              'budget': budgets,
                              'synopsis': plot_synopsis,
                              'pro. comp': production_companies,
                              'spec. eff': spec_eff_comps,
                                'gross': gross,
                              'stars': stars,
                        'director': directors,
                        'writer': screen_writers,
                             'metascore': metascores,
                              'genre': genres,
                              'mpaa': mpaa_ratings,
                              'imdb': imdb_ratings
                             })

print(movie_ratings.info())
movie_ratings.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 15 columns):
movie        23 non-null object
length       23 non-null object
date         23 non-null object
budget       23 non-null object
synopsis     23 non-null object
pro. comp    23 non-null object
spec. eff    23 non-null object
gross        23 non-null object
stars        23 non-null object
director     23 non-null object
writer       23 non-null object
metascore    23 non-null int64
genre        23 non-null object
mpaa         23 non-null object
imdb         23 non-null float64
dtypes: float64(1), int64(1), object(13)
memory usage: 2.8+ KB
None


,movie,length,date,budget,synopsis,pro. comp,spec. eff,gross,stars,director,writer,metascore,genre,mpaa,imdb
0,Mortal Engines,128 min,14 December 2018,100000000,The film starts with the voice of Shrike (Step...,"[Media Rights Capital (MRC), Scholastic Produc...","[Double Negative (DNEG), Halo VFX, Argon Effec...",15951040,"[Hera Hilmar, Robert Sheehan, Hugo Weaving, Ji...",Christian Rivers,"[Jonathan Kasdan, Lawrence Kasdan, George Lucas]",44,"Action, Adventure, Fantasy",PG-13,6.2
1,Deadpool 2,119 min,18 May 2018,110000000,After successfully working as the mercenary De...,"[Donners' Company, Kinberg Genre, Marvel Enter...","[Double Negative (DNEG), Halo VFX, Argon Effec...",324591735,"[Ryan Reynolds, Josh Brolin, Morena Baccarin, ...",David Leitch,"[Jonathan Kasdan, Lawrence Kasdan, George Lucas]",66,"Action, Adventure, Comedy",R,7.8
2,Crazy Rich Asians,120 min,15 August 2018,30000000,"London, England 1995Eleanor Young (Michelle Ye...","[Warner Bros., SK Global, Starlight Culture En...","[Double Negative (DNEG), Halo VFX, Argon Effec...",174532921,"[Constance Wu, Henry Golding, Michelle Yeoh, G...",Jon M. Chu,"[Jonathan Kasdan, Lawrence Kasdan, George Lucas]",74,"Comedy, Romance",PG-13,7.0
3,Vox Lux,114 min,20 December 2018,null,It looks like we don't have a Synopsis for thi...,"[Bold Films, Killer Films, Andrew Lauren Produ...","[Double Negative (DNEG), Halo VFX, Argon Effec...",727119,"[Natalie Portman, Jude Law, Stacy Martin, Jenn...",Brady Corbet,"[Jonathan Kasdan, Lawrence Kasdan, George Lucas]",68,"Drama, Music",R,6.2
4,Second Act,103 min,21 December 2018,16000000,Maya Vargas (Jennifer Lopez) is about to turn ...,[STX Entertainment],"[Double Negative (DNEG), Halo VFX, Argon Effec...",39282227,"[Jennifer Lopez, Vanessa Hudgens, Leah Remini,...",Peter Segal,"[Jonathan Kasdan, Lawrence Kasdan, George Lucas]",46,"Comedy, Drama, Romance",PG-13,5.9
5,Lords of Chaos,118 min,29 March 2019,null,It looks like we don't have a Synopsis for thi...,"[4 1/2 Film, Chimney, Eleven Arts, Insurgent M...","[Double Negative (DNEG), Halo VFX, Argon Effec...",219846,"[Rory Culkin, Emory Cohen, Jack Kilmer, Sky Fe...",Jonas Åkerlund,"[Jonathan Kasdan, Lawrence Kasdan, George Lucas]",47,"Biography, Drama, Horror",R,6.4
6,Widows,129 min,16 November 2018,42000000,The film opens by introducing the main charact...,"[Regency Enterprises, See-Saw Films, Film4, La...","[Double Negative (DNEG), Halo VFX, Argon Effec...",42402632,"[Viola Davis, Michelle Rodriguez, Elizabeth De...",Steve McQueen,"[Jonathan Kasdan, Lawrence Kasdan, George Lucas]",84,"Crime, Drama, Thriller",R,7.1
7,Moonlight,111 min,18 November 2016,1500000,This film is told through three different poin...,"[A24, PASTEL, Plan B Entertainment]","[Double Negative (DNEG), Halo VFX, Argon Effec...",27850912,"[Mahershala Ali, Naomie Harris, Trevante Rhode...",Barry Jenkins,"[Jonathan Kasdan, Lawrence Kasdan, George Lucas]",99,Drama,R,7.4
8,Ocean's 8,110 min,8 June 2018,70000000,The film starts with Debbie Ocean (Sandra Bull...,"[Warner Bros., Village Roadshow Pictures, Rahw...","[Double Negative (DNEG), Halo VFX, Argon Effec...",139377762,"[Sandra Bullock, Cate Blanchett, Anne Hathaway...",Gary Ross,"[Jonathan Kasdan, Lawrence Kasdan, George Lucas]",61,"Action, Comedy, Crime",PG-13,6.2
9,The Greatest Showman,105 min,20 December 2017,84000000,"The film opens with Phineas Taylor ""P.T."" Barn...","[Chernin Entertainment, TSG Entertainment, Twe...","[Double Negative (DNEG), Halo VFX, Argon Effec...",174340174,"[Hugh Jackman, Michelle Williams, Zac Efron, Z...",Michael Gracey,"[Jonathan Kasdan, Lawrence Kasdan, George Lucas]",48,"Biography, Drama, Musical",PG,7.6


In [248]:
url = "https://www.imdb.com/title/tt1727824/companycredits"
response = get(url)

effects_html = BeautifulSoup(response.text, 'html.parser')

effects_comps = []
comps_x = effects_html.find("h4", {"id": "specialEffects"}).findNext('ul').findAll('li')
for comp in comps_x:
    effects_comps.append(comp.a.text)
effects_comps

['Double Negative (DNEG)',
 'Halo VFX',
 'Argon Effects',
 'Clear Angle Studios',
 'Centroid Motion Capture',
 'The Visual Effects Company',
 'Fangs FX']